In [ ]:
# for determining whether genes from a reference database occur in a contig

refgenome = 'E.coli.K12'
contig = 'Sample.Genome.Contig.'
passnum = 1
limit = 0.95          # the fraction of bases that must match
#step = the number of genes in a step size between search. It is assigned just prior to the first search routine 
#numzoom = the number of genes in the drill-down.  It is defined just above the second search routine
#import requests
#!pip install fuzzywuzzy
#from fuzzywuzzy import fuzz
import math
from google.colab import drive
drive.mount('/content/gdrive')
contigname = '/content/gdrive/Shared drives/2.Python/Genomics/Data Files/' + contig + str(passnum)+ '.txt'
genesname =  '/content/gdrive/Shared drives/2.Python/Genomics/Data Files/' + refgenome + '.Gene.Database.txt'
outfile =    '/content/gdrive/Shared drives/2.Python/Genomics/Data Files/' + contig + refgenome + '.MappedGenes.txt'

with open(contigname, 'r') as f:
  contigfile = str(f.read())
  #print(repr(data)[:200])
  #print(repr(data)[-200:])
f.close()
contig = contigfile.replace('"', '').replace('b', '')
print('Contig length =', len(contig))
with open(genesname, 'r') as g:
  genesfile = str(g.read())
  #print(repr(data)[:200])
  #print(repr(data)[-200:])
g.close()
genelist = genesfile.split('\n')
genelist.pop()
print('The database has', len(genelist), 'genes.')

genes = []
for x in range(len(genelist)):
  row = genelist[x].split(',')
  genes.append(row)

#print(genes[0])
#print(contig[:100])

def rev(string1):  
  string2 =''
  for x in string1:
    if   x == 'c': string2 += 'g'
    elif x == 'g': string2 += 'c'
    elif x == 't': string2 += 'a'
    elif x == 'a': string2 += 't'
  string3 = string2[::-1]  
  return(string3)

def match(string1, string2):
  sum =0
  for x in range(len(string1)):
    if string1[x] == string2[x]:
      sum +=1
  ratio = sum / len(string1)
  return(ratio)

hits =[]
stop = False
step = math.floor(len(contig) / 1500 )   # determines how many genes between the initial search
print('Size of the step in the inital pass through the database =', step)
numzoom = math.ceil(len(contig) / 600 )   # determines how many genes to drill down on
print('Number of genes searched on either side of the initial hit =', numzoom)
print()
for x in range(3000, len(genes)):
  if x % step == 0 and stop == False :
    if len(genes[x][3]) in range (50, len(contig) -50):
      #print(genes[x][0])
      query = genes[x][3][9:-9]
      revsq = rev(genes[x][3][9:-9])
      sum =0
      if x % 100 == 0: print('Number of genes searched thus far...', x)
      for y in range(len(contig) - len(query)):
        window = contig[y: y + len(query)]
        if match(query, window) >= limit or match(revsq, window) >= limit:  # can use fuzz.ratio instead of match
        #if fuzz.ratio(query, window) >= limit or fuzz.ratio(revsq, window) >= limit:  # can use fuzz.ratio instead of match
          hits.append( ['Gene #', x, genes[x][0], genes[x][1] ]  )
          print('Gene #', x, ',', genes[x][0], ', at position:', genes[x][1])
          stop = True
          new = x
          break

for z in range(new - numzoom, new + numzoom):
  if z != new:
    if len(genes[z][3]) in range (50, len(contig) -50):
      query = genes[z][3][9:-9]
      revsq = rev(genes[z][3][9:-9])
      sum =0
      for y in range(len(contig) - len(query)):
        window = contig[y: y + len(query)]
        if match(query, window) >= limit or match(revsq, window) >= limit:  # can use fuzz.ratio instead of match
        #if fuzz.ratio(query, window) >= limit or fuzz.ratio(revsq, window) >= limit:  # can use fuzz.ratio instead of match
#          hits.append( ['Gene number: ',z,  genes[z][:3] ] )
          hits.append( ['Gene #', z, genes[z][0], genes[z][1] ]  )
          print('Gene #', z, ',', genes[z][0], ', at position:', genes[z][1])
          break

print()
hits.sort(key = lambda x: x[1]) 
outlist =[]
for x in range(len(hits)):
  outlist.append( [ hits[x][0] +','+ str(hits[x][1]) +','+ str(hits[x][2]) +','+ str(hits[x][3]) +'\n' ] )
  
print()
with open(outfile, 'w') as h:
  for x in range(len(hits)):
    outline = hits[x][0] +','+ str(hits[x][1]) +','+ str(hits[x][2]) +','+ str(hits[x][3])
    h.write(outline + '\n')
    print(outline)
h.close()
print()
print('Hits for reference genome,', refgenome, 'with a match score >=', limit, '. These gene hits are not saved to a file.')
print('Done.')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Contig length = 6780
The database has 4175 genes.
Size of the step in the inital pass through the database = 4
Number of genes searched on either side of the initial hit = 12

Number of genes searched thus far... 3000
Gene # 3020 , rbfA , at position: 3312777
Gene # 3021 , infB , at position: 3313342
Gene # 3022 , nusA , at position: 3316039
Gene # 3023 , rimP , at position: 3317554


Gene #,3020,rbfA,3312777
Gene #,3021,infB,3313342
Gene #,3022,nusA,3316039
Gene #,3023,rimP,3317554

Hits for reference genome, E.coli.K12 with a match score >= 0.95 . These gene hits are not saved to a file.
Done.
